In [ ]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import scale

from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

############################################################################################
# vae: build a variational autoencoder using the input data
# 
# This function is heavily derived from - 
# 
# Inputs:
#   - counts_mat: matrix with gene names as counts and individual cells are rows
#   - loss_fun: loss function for autoencoder
#   - latent_dim: number of latent dimensions to project data into
#   - intermediate_dim: number of dimensions in the hidden layer
#   - batch_size: batch size used for training autoencoder
#   - epochs: epochs for training autoencoder
#   - nbshape: shape parameter for negative binomial loss. Defaults to 1
# 
# Outputs:
#   - x_encoded: Encoded transformation of input
#   - z_mean: parameter for the mean of distribution in the latent space
#
############################################################################################

def vae(counts_mat, loss_fun, latent_dim, intermediate_dim, 
        batch_size, epochs, epsilon_std, nbshape=1):
            
    x_train = preprocess(counts_mat, loss_fun)
    original_dim = x_train.shape[1]
    
    # encoder network: map inputs to latent distribution parameters
    x = Input(shape=(original_dim,))
    h = Dense(intermediate_dim, activation='relu')(x)
    z_mean = Dense(latent_dim)(h)
    z_log_var = Dense(latent_dim)(h)

    
    # use 'sampling' function to sample new similar points from the latent space
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    
    # map these sampled latent points back to reconstructed inputs 
    # i.e. a sort of decoder network
    decoder_h = Dense(intermediate_dim, activation='relu')
    decoder_mean = Dense(original_dim, activation='sigmoid')
    h_decoded = decoder_h(z)
    x_decoded_mean = decoder_mean(h_decoded)

    # build end-to-end autoencoder
    vae_model = Model(x, x_decoded_mean)
    

    # build encoder, from inputs to latent space
    encoder = Model(x, z_mean)
    

    # Train the model using the end-to-end model, with a custom loss function: 
    # the sum of a reconstruction term, and the KL divergence regularization term.
    # Use different user defined loss functions to generate reconstruction loss
    vae_loss = vae_loss(loss_fun, x, x_decoded_mean, nbshape, z_log_sigma, z_mean)
    vae_model.compile(optimizer='rmsprop', loss=vae_loss)
    vae_model.summary()
    
    
    # fit the model with the given data
    vae_model.fit(x_train, x_train, 
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2)
    

    # project/encode inputs on the latent space
    encoder = Model(x_train, z_mean)
    x_encoded = encoder.predict(x_train, batch_size=batch_size)
    
    
    return x_encoded, z_mean

############################################################################################
# preprocess: preprocess counts matrix to make it suitable for particular loss function
#
# Inputs:
#   - Y: matrix with gene names as counts and individual cells are rows
#   - loss_fun: loss function for autoencoder
#
# Outputs:
#   - x_train: preprocessed matrix of counts for training the autoencoder
############################################################################################

def preprocess(counts_mat, loss_fun):
    if loss_fun == 'gaussian':
        x_train = np.log2(1 + (counts_mat.T/np.sum(counts_mat, axis=1)).T * 1e+6) # log2(1+Y/rowSums(Y)*1e6)
        x_train = scale(x_train, axis=1, with_mean=True, with_std=True, copy=True) # scale_center(Y)
        
    elif loss_fun == 'bernoulli':
        x_train = counts_mat
        x_train[x_train > 0] = 1 # int(Y>0)
        
    else:
        x_train = counts_mat
    
    return x_train

############################################################################################
# sampling: sample encoded values of the input using latent space parameters
#
# Inputs:
#   - args: mean and variance parameters of the latent space distribution
# 
# Outputs:
#   - encoded points sampled from the latent space distribution
############################################################################################

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

############################################################################################
# vae_loss: compute loss of variational encoder adding reconstruction & regularization loss
# 
# Inputs:
#   - loss_fun: loss function for autoencoder
#   - x: preprocessed input to the autoencoder
#   - x_decoded_mean: mean of the x decoded by the autoencoder
#   - nbshape: shape parameter for negative binomial loss. Defaults to 1
# 
# Outputs:
#   - loss: loss from the autoencoding
############################################################################################

def vae_loss(loss_fun, x, x_decoded_mean, nbshape, z_log_sigma, z_mean):
    if loss_fun == 'poisson':
        reconstruction_loss = objectives.poisson(x, x_decoded_mean)
        
    elif loss_fun == 'negative_binomial':
        reconstruction_loss = negative_binomial_loss(x, x_decoded_mean, nbshape)
        
    elif loss_fun == 'gaussian':
        reconstruction_loss = gaussian_loss(x, x_decoded_mean)
        
    elif loss_fun == 'bernoulli':
        reconstruction_loss = bernoulli_loss(x, x_decoded_mean)
        
    else:
        reconstruction_loss = objectives.binary_crossentropy(x, x_decoded_mean) # defaults to cross entropy
    
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    
    return reconstruction_loss + kl_loss

In [1]:
# These next cells show that preprocessing is working as expected

import numpy as np
from sklearn.preprocessing import scale

# lets see what the input matrix looks like
np.array([[0, 1, 3, 4], [0, 5, 1, 4]])


array([[0, 1, 3, 4],
       [0, 5, 1, 4]])

In [2]:
# preprocessing for gaussian
x = np.array([[0, 1, 3, 4], [0, 5, 1, 4]])
y = np.log2(1 + (x.T/np.sum(x, axis=1)).T * 1e+6)
y = scale(y, axis=1, with_mean=True, with_std=True, copy=True )

y

array([[-1.7242733 ,  0.42319561,  0.62421885,  0.67685884],
       [-1.72094179,  0.68567751,  0.39051082,  0.64475346]])

In [3]:
# preprocessing for bernoulli
x = np.array([[0, 1, 3, 4], [0, 5, 1, 4]])
y = x
y[y>0] = 1

y

array([[0, 1, 1, 1],
       [0, 1, 1, 1]])